# Tribolium embryo morphometry over time in Napari
Authors: Robert Haase, Daniela Vorkel, 2020

This is the pyclesperanto version of a workflow earlier [published for clij2](https://clij.github.io/clij2-docs/md/tribolium_morphometry/). 
[ImageJ Macro original](https://github.com/clij/clij2-docs/tree/master/src/main/macro/tribolium_morphometry.ijm)

This script is an example of heavy GPU-accelerated processing. It is recommended to use a dedicated
graphics card with at least 8 GB of GDDR6 memory. Otherwise, it may be quite slow.

Let's start by checking that pyclesperanto is installed and which GPU it uses.

In [1]:
import pyclesperanto_prototype as cle
import numpy as np

# show all graphics cards
#print(cle._tier0._pycl.filter_devices())

# show only GPU devices
print(cle._tier0._pycl.filter_devices(dev_type='gpu'))

# selecting an Nvidia RTX
cle.select_device("Quadro M2200")
print("Using OpenCL device " + cle.get_device().name)

[<pyopencl.Device 'Quadro M2200' on 'NVIDIA CUDA' at 0x18aa0823f90>, <pyopencl.Device 'Intel(R) HD Graphics 530' on 'Intel(R) OpenCL' at 0x18aa0b42dd0>]
Using OpenCL device Quadro M2200


In [2]:
%gui qt

## Load a data set
The dataset shows a *Tribolium castaneum* embryo, imaged by a custom light sheet microscope, at a wavelength of 488nm (Imaging credits: Daniela Vorkel, Myers lab, MPI CBG). 
The data set has been resampled to a voxel size of 1x1x1 microns. The embryo expresses nuclei-GFP. We will use the dataset to detect nuclei and to generate an estimated cell-segmentation.

All processing steps are performed in 3D space.

In [3]:
from skimage.io import imread
from aicspylibczi import CziFile
import imgfileutils as imf

#timelapse = imread('C:/structure/data/clincubator_data/Lund_18.0_22.0_Hours-iso.tif')
# print out the spatial dimensions of the image
#print(timelapse.shape)


filename = r"C:\Users\m1srh\OneDrive - Carl Zeiss AG\Testdata_Zeiss\Castor\Z-Stack_DCV\CellDivision_T=15_Z=20_CH=2_DCV.czi"

# get the metadata
md, addmd = imf.get_metadata(filename)

# get czi object
czi = CziFile(filename)
timelapse, shp = czi.read_image(S=0, C=0)
timelapse = np.squeeze(timelapse)
print(timelapse.shape)

Detected Image Type (based on extension):  czi


C:\ProgramData\Anaconda3\envs\imageanalysis\lib\site-packages\aicsimageio\transforms.py:178: UserWarning: Data has dimension B with depth 1, assuming B=0 is the desired value, if not the case specify B=x where x is an integer, list, tuple, range, or slice.
  f"Data has dimension {dim} with depth {data.shape[dim_index]}, "


Trying to extract Scene and Well information if existing ...
No valid Scene or Well information found: 'S'
Scales will not be rounded.
(15, 20, 700, 700)


In [4]:
def process_image(image):
    import time

    start_time = time.time()
    
    # push image to GPU memory and show it
    gpu_input = cle.push_zyx(image)
    # print(gpu_input)
    
    # gaussian blur
    sigma = 2.0
    gpu_blurred = cle.gaussian_blur(gpu_input, sigma_x=sigma, sigma_y=sigma, sigma_z=sigma)

    # detect maxima
    gpu_detected_maxima = cle.detect_maxima_box(gpu_blurred)
    
    # threshold
    threshold = 300.0
    gpu_thresholded = cle.greater_constant(gpu_blurred, constant=threshold)

    # mask
    gpu_masked_spots = cle.mask(gpu_detected_maxima, gpu_thresholded)

    # label spots
    gpu_labelled_spots = cle.connected_components_labeling_box(gpu_masked_spots)
    # show_labels(gpu_labelled_spots)
    
    number_of_spots = int(cle.maximum_of_all_pixels(gpu_labelled_spots))
    # print("Number of detected spots: " + str(number_of_spots))
    
    # label map closing
    number_of_dilations = 10
    flip = cle.create_like(gpu_labelled_spots)
    flop = cle.create_like(gpu_labelled_spots)
    flag = cle.create([1,1,1])
    cle.copy(gpu_labelled_spots, flip)

    for i in range (0, number_of_dilations) :
        cle.onlyzero_overwrite_maximum_box(flip, flag, flop)
        cle.onlyzero_overwrite_maximum_diamond(flop, flag, flip)

    # erode labels
    flap = cle.greater_constant(flip, constant=1)
    number_of_erosions = 4
    for i in range(0, number_of_erosions):
        cle.erode_box(flap, flop)
        cle.erode_box(flop, flap)

    gpu_labels = cle.mask(flip, flap)
    
    # get result back from GPU as numpy array
    result = cle.pull_zyx(gpu_labels)
        
    print("Processing took " + str(time.time() - start_time) + " s")

    return result

In [5]:
from skimage import data
import napari
viewer = napari.Viewer()

In [6]:
def get_scalefactor(metadata):

    # set default scale factor to 1.0
    scalefactors = [1, 1, 1]

    # get the factor between XY scaling
    scalefactors[0] = metadata['XScale'] / metadata['YScale']
    scalefactors[1] = metadata['XScale'] / metadata['YScale']
    scalefactors[2] = metadata['ZScale'] / metadata['YScale']

    return scalefactors

# voxel size z,y,x
#calibration = [1, 1, 1]

sf = get_scalefactor(md)
print(sf)


# convenience function for visualisation
def show(image):
    viewer.add_image(image, scale=[sf[2], sf[1], sf[0]])
    
def show_labels(labels):
    viewer.add_labels(labels, scale=[sf[2], sf[1], sf[0]])

[1.0, 1.0, 3.5329184140969163]


In [7]:
# adapted from: https://github.com/tlambert03/napari-dask-example/blob/master/dask_napari.ipynb
import dask
import dask.array as da

# create dask stack of lazy image readers
lazy_process_image = dask.delayed(process_image)  # lazy reader

lazy_arrays = [lazy_process_image(timelapse[n]) for n in range(0, timelapse.shape[0])]

dask_arrays = [
    da.from_delayed(lazy_array, shape=timelapse[0].shape, dtype=timelapse[0].dtype)
    for lazy_array in lazy_arrays
]
# Stack into one large dask.array
dask_stack = da.stack(dask_arrays, axis=0)
dask_stack

dask.array<stack, shape=(15, 20, 700, 700), dtype=uint16, chunksize=(1, 20, 700, 700), chunktype=numpy.ndarray>

In [8]:
print(timelapse.shape[0])

15


In [9]:
#show(timelapse)
#show_labels(dask_stack)

In [ ]:
from napari.utils import nbscreenshot
nbscreenshot(viewer)